<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/quantum_metric_kernel_sweep_simple_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn matplotlib

In [ ]:
#!/usr/bin/env python3
"""
quantum_metric_kernel_sweep_simple.py

End-to-end sweep (no noise) for 4-bit parity:

1) Builds parity datasets at various class-imbalance ratios
2) Runs analytic CNOT and variational QNN baselines
3) Trains an interleaved feature-map metric learner via centered alignment (Adam)
4) Sweeps ansatz depths and imbalance ratios
5) Evaluates learned kernel with SVM CV
6) Plots alignment curves and CV accuracies
"""

import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp, grad
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# 1) Data utilities
# -----------------------------------------------------------------------------
def build_parity_dataset(n_bits, imbalance_ratio=1.0, seed=0):
    """
    Generate up to 2^n_bits samples of n-bit parity,
    subsampling the odd class to achieve the given even:odd ratio.
    """
    rng = np.random.RandomState(seed)
    X_bits = np.array(list(itertools.product([0, 1], repeat=n_bits)))
    y_bits = np.sum(X_bits, axis=1) % 2

    if imbalance_ratio < 1.0:
        idx_even = np.where(y_bits == 0)[0]
        idx_odd  = np.where(y_bits == 1)[0]
        target_odd = int(len(idx_even) * (1 - imbalance_ratio) / imbalance_ratio)
        idx_odd_sub = rng.choice(idx_odd, size=target_odd, replace=False)
        keep = np.hstack([idx_even, idx_odd_sub])
        X_bits, y_bits = X_bits[keep], y_bits[keep]

    # Scale bits to [0, π] for AngleEmbedding
    X_feat = X_bits.astype(float) * np.pi
    y_signed = 1 - 2 * y_bits  # +1 even, -1 odd
    return X_bits, X_feat, y_bits, y_signed

# -----------------------------------------------------------------------------
# 2) Analytic CNOT baseline
# -----------------------------------------------------------------------------
def analytic_baseline(X_bits, y_bits):
    n_bits = X_bits.shape[1]
    dev = qml.device("default.qubit", wires=n_bits + 1)

    @qml.qnode(dev)
    def circ(x):
        for i, b in enumerate(x):
            if b:
                qml.PauliX(wires=i)
        for i in range(n_bits):
            qml.CNOT(wires=[i, n_bits])
        return qml.expval(qml.PauliZ(wires=n_bits))

    preds = [0 if circ(x) > 0 else 1 for x in X_bits]
    return accuracy_score(y_bits, preds)

# -----------------------------------------------------------------------------
# 3) Variational QNN baseline (5-fold CV)
# -----------------------------------------------------------------------------
def variational_qnn_cv(X_feat, y_bits, y_signed, qnn_epochs, qnn_lr, seed=0):
    n_bits = X_feat.shape[1]
    dev = qml.device("default.qubit", wires=n_bits)
    kf  = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    @qml.qnode(dev, interface="autograd")
    def qnn_circuit(params, x):
        for i, angle in enumerate(x):
            qml.RY(angle, wires=i)
        for i in range(n_bits):
            qml.RY(params[i], wires=i)
        for i in range(n_bits - 1):
            qml.CNOT(wires=[i, i + 1])
        return qml.expval(qml.PauliZ(wires=n_bits - 1))

    scores = []
    for tr, te in kf.split(X_feat, y_bits):
        params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
        opt    = qml.GradientDescentOptimizer(stepsize=qnn_lr)
        for _ in range(qnn_epochs):
            def cost(p):
                preds = [qnn_circuit(p, X_feat[i]) for i in tr]
                return pnp.mean((pnp.array(preds) - y_signed[tr]) ** 2)
            params = opt.step(cost, params)

        preds_test = [qnn_circuit(params, X_feat[i]) for i in te]
        bits_pred  = [0 if v > 0 else 1 for v in preds_test]
        scores.append(accuracy_score(y_bits[te], bits_pred))

    return np.mean(scores), np.std(scores)

# -----------------------------------------------------------------------------
# 4) Interleaved feature map, kernel builder, and alignment
# -----------------------------------------------------------------------------
def feature_map_kernel(params, X_feat, y_signed, num_layers):
    n_bits = X_feat.shape[1]
    dev = qml.device("default.qubit", wires=n_bits)

    @qml.qnode(dev, interface="autograd")
    def fmap(p, x):
        offset = 0
        for _ in range(num_layers):
            # data reupload
            qml.templates.AngleEmbedding(x, wires=range(n_bits))
            # trainable rotations
            for i in range(n_bits):
                a, b, c = p[offset + 3*i: offset + 3*i + 3]
                qml.Rot(a, b, c, wires=i)
            offset += 3 * n_bits
            # ring entanglement
            for i in range(n_bits):
                qml.CNOT(wires=[i, (i + 1) % n_bits])
        # final re-encoding
        qml.templates.AngleEmbedding(x, wires=range(n_bits))
        return qml.state()

    # Build fidelity-squared Gram matrix
    states = pnp.stack([fmap(params, x) for x in X_feat])
    re, im  = pnp.real(states), pnp.imag(states)
    real_ov = re @ re.T + im @ im.T
    imag_ov = re @ im.T - im @ re.T
    K = real_ov**2 + imag_ov**2

    # Centered kernel-target alignment
    N  = K.shape[0]
    H  = pnp.eye(N) - pnp.ones((N, N)) / N
    Kc = H @ K @ H
    T  = pnp.outer(y_signed, y_signed)
    align = pnp.sum(Kc * T) / (pnp.linalg.norm(Kc) * pnp.linalg.norm(T))

    return K, align

# -----------------------------------------------------------------------------
# 5) Metric learning via Adam
# -----------------------------------------------------------------------------
def train_metric_alignment(
    X_feat, y_signed, num_layers,
    metric_lr, metric_steps, seed=0
):
    dim    = num_layers * 3 * X_feat.shape[1]
    params = pnp.random.randn(dim, requires_grad=True) * 0.1
    opt    = qml.AdamOptimizer(stepsize=metric_lr)
    history = []

    for step in range(metric_steps):
        def loss_fn(p):
            _, a = feature_map_kernel(p, X_feat, y_signed, num_layers)
            return -a

        grads     = grad(loss_fn)(params)
        grad_norm = float(pnp.linalg.norm(grads).item())
        params, loss_val = opt.step_and_cost(loss_fn, params)

        if step % 10 == 0 or step == metric_steps - 1:
            _, align_val = feature_map_kernel(params, X_feat, y_signed, num_layers)
            history.append((step,
                            float(loss_val.item()),
                            grad_norm,
                            float(align_val.item())))
    return params, history

# -----------------------------------------------------------------------------
# 6) SVM CV on precomputed kernel
# -----------------------------------------------------------------------------
def svm_precomputed_cv(K, y_bits, svm_C, seed=0):
    kf     = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    scores = []
    for tr, te in kf.split(K, y_bits):
        svc = SVC(kernel="precomputed", C=svm_C)
        svc.fit(K[np.ix_(tr, tr)], y_bits[tr])
        preds = svc.predict(K[np.ix_(te, tr)])
        scores.append(accuracy_score(y_bits[te], preds))
    return np.mean(scores), np.std(scores)

# -----------------------------------------------------------------------------
# 7) Main sweep & plotting
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    # Settings
    n_bits       = 4
    qnn_epochs   = 30
    qnn_lr       = 0.4
    metric_steps = 200
    metric_lr    = 0.1
    svm_C        = 1.0
    seed         = 42

    depths           = [1, 3, 5]
    imbalance_ratios = [1.0, 0.75, 0.50]

    results = []

    for ratio in imbalance_ratios:
        X_bits, X_feat, y_bits, y_signed = build_parity_dataset(
            n_bits, ratio, seed
        )

        # Baselines
        acc_cnot   = analytic_baseline(X_bits, y_bits)
        qnn_mean, qnn_std = variational_qnn_cv(
            X_feat, y_bits, y_signed, qnn_epochs, qnn_lr, seed
        )
        print(f"\nImbalance {ratio:.0%}: CNOT={acc_cnot:.3f}, QNN={qnn_mean:.3f}±{qnn_std:.3f}")

        for depth in depths:
            params_k, history = train_metric_alignment(
                X_feat, y_signed, depth, metric_lr, metric_steps, seed
            )

            K_learn, align_learn = feature_map_kernel(
                params_k, X_feat, y_signed, depth
            )
            svm_mean, svm_std = svm_precomputed_cv(K_learn, y_bits, svm_C, seed)

            results.append({
                "ratio": ratio,
                "depth": depth,
                "align": align_learn,
                "svm_mean": svm_mean,
                "svm_std": svm_std,
                "history": history
            })

            print(
                f"  depth={depth:>1d} | align={align_learn:.3f} "
                f"| SVM={svm_mean:.3f}±{svm_std:.3f}"
            )

    # Plot alignment vs step for each depth (balanced data)
    plt.figure(figsize=(6, 4))
    for res in results:
        if res["ratio"] == 1.0:
            steps, _, _, aligns = zip(*res["history"])
            plt.plot(steps, aligns, label=f"layers={res['depth']}")
    plt.xlabel("Training Step")
    plt.ylabel("Centered Alignment")
    plt.title("Alignment vs Step (balanced)")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Plot SVM CV accuracy vs imbalance for each depth
    plt.figure(figsize=(6, 4))
    for depth in depths:
        means = [
            r["svm_mean"] for r in results if r["depth"] == depth
        ]
        plt.plot(imbalance_ratios, means, marker="o", label=f"layers={depth}")
    plt.xlabel("Class-imbalance ratio (even)")
    plt.ylabel("SVM CV Accuracy")
    plt.title("Imbalance Sensitivity")
    plt.legend()
    plt.tight_layout()
    plt.show()